In [1]:
from __future__ import division, absolute_import, print_function
import sys
if sys.version_info < (3,):
    range = xrange
import os
from pylab import *  # for plotting
from graph_tool.all import *
import random as rd
import itertools #Biblioteca para geração da lista de elementos repetidos

In [7]:
#---------------------------------------------Parametros iniciais--------------------------------------------------

#m0: número de sítios iniciais da rede
m0 = 4
#m: número de ligações que os novos nós devem fazer
m = 2 
#t: por quanto tempo o processo será implementando (gerando novos sítios)
t = 10000
#N: número de sítios da rede
N = m0 + t

## Número de sítios da rede

Logo de começo devemos restringir o valor de $m$ com relação a $m_0$, fazendo

<center> $ m \le m_0 $, (1) </center>

essa restrição evita ligações em loop do novo sítio. Nós temos que o número de sítos final da rede será dada por

<center> $N = m_0 + t$, (2) </center>

para cada instante de tempo $t$, haverá $m_0+t$ sítios. Com número de ligações dadas por

<center> $l = m_0+mt$, (3)</center>

pois cada um dos $t$ sítios terão $m$ ligações. Para sabermos quanto devemos utilizar no tempo para um número $N$ de sítios (quantas interações devemos fazer), podemos isolar o $t$ em (2),obtendo

<font size="3"><center> $N-m_0 = t$.  </center></font>

A probabilidade de ligação de um novo sítio $i$ é dada por

<font size="3"><center> $p_i = \frac{k_i}{\sum_i k_i}$ </center></font>,

com a distribuição dos graus dados por

<font size="3"><center> p(k) = $\frac{2m_0(m_0+1)}{k(k+1)(k+2)}$ </center></font>



## 2. Algoritmo para rede inicial

Tomando a rede inicial com $m_0$ sítios totalmente conectada (sítios igualmente provaveis inicialmente), teremos a matriz de adjacência dada por

\begin{equation}
A = 
\begin{pmatrix}
0 & 1 & 1 & \cdots & 1\\
1 & 0 & 1 & \cdots & 1\\
1 & 1 & 0 & \cdots & 1\\
1 & 1 & 1 & \ddots & 1\\
1 & 1 & 1 & \cdots & 0\\
\end{pmatrix}
_{(m_0,m_0)},
\end{equation}

A rede totalmente conectada, possuí todos elementos fora da coluna principal igual a 1. Para isso, podemos expressar um algoritmo primeiro fixando não-ligação para $i=j$ (sem loops), e logo em seguinda tomando a condição de que $j\le i$, para pegar apenas os elementos do triângulo superior da matriz.

In [3]:
g = Graph(directed=False)

m0 = 4  #Número de sítios iniciais;
m = 3  #Número de conexões dos novos sítios;
N = 10 #Número de sítios final;
t = N - m0 #Número de passos, lembrando que após t instantes de tempo, teremos N = m0 + t sítios;

vlist = g.add_vertex(m0) #Add m0 elements de sítios em g;

ver = np.zeros(N) #Guardará os sítios num array;

for i in range(m0):
    ver[i] = int(g.vertex(i))
    for j in range(i+1,m0):
        g.add_edge(g.vertex(i),g.vertex(j))


#Definindo  os arrays que guardará o grau, sítios e conjunto de graus e sítios da rede;
DegreeFN = [] #Graus da nossa rede totalmente conectada terá m*t+m0 conexões FN: final nodes;

VertexF = [] #Número total de sítios;

K = [] #Lista que guardará os "k" valores dos "i" sítios;
K_supli = [] #Lista Suplementar, que juntará os novos elementos ao K;
K_filter = [] #Lista que será filtrada para ser usada dentro do loop;
#-----------------------------------------------------

#Gerando as listas com os sítios e ligações
for i in range(m0):
    A = int(g.vertex(i).out_degree())
    B = int(g.vertex(i))
    DegreeFN.append(A)
    VertexF.append(B)
    
#------------------------------------------------
for i in range(m0):
    K_supli = list(itertools.repeat(VertexF[i], DegreeFN[i]))
    K += K_supli
    K_filter += K_supli

print(K)
print(g.get_vertices())


[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3]
[0 1 2 3]


In [4]:
m0 = 4  #Número de sítios iniciais;
m = 3  #Número de conexões dos novos sítios;
N = 10

for i in range(m0,N):
    v = g.add_vertex()
    for j in range(m):
        j = randint(0, len(K_filter))
        u = g.vertex(K_filter[j])
        L = g.add_edge(v,u)
        #DegreeFN.append(int(u))
        #VertexF.append(int(u.out_degree()))
        #K_supli = list(itertools.repeat(int(u), int(u.out_degree())))
        #K += K_supli
        K_filter = list(filter(lambda val: val !=  u, K_filter))
        print(L)
    
    DegreeFN.append(int(g.vertex(i).out_degree()))
    VertexF.append(int(g.vertex(i)))
    print(DegreeFN)
    print(VertexF)
    K_supli = list(itertools.repeat(VertexF[i], DegreeFN[i]))
    K += K_supli
    K_filter = K

print(K)
print(K_filter)



(4, 2)
(4, 3)
(4, 0)
[3, 3, 3, 3, 3]
[0, 1, 2, 3, 4]
(5, 4)
(5, 1)
(5, 2)
[3, 3, 3, 3, 3, 3]
[0, 1, 2, 3, 4, 5]
(6, 5)
(6, 2)
(6, 3)
[3, 3, 3, 3, 3, 3, 3]
[0, 1, 2, 3, 4, 5, 6]
(7, 3)
(7, 1)
(7, 6)
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 1, 2, 3, 4, 5, 6, 7]
(8, 7)
(8, 1)
(8, 2)
[3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
(9, 6)
(9, 4)
(9, 2)
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9]
[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9]


In [13]:
for i in range(3):
    j = randint(0, len(K_filter))
    u = g.vertex(K_filter[j])
    L = g.add_edge(v,u)
    K_filter = list(filter(lambda val: val !=  u, K_filter))
    
K_filter = K
    
    


In [89]:
a = 2

In [10]:
#m0: número de sítios' iniciais;
def barabasi_network(m0,m,N,s):
    seed(s)
    g = Graph(directed=False) #Grafo sem direção
    
    #S = g.new_vertex_property("double")
    #L = g.new_edge_property("double")
    
    vlist = g.add_vertex(m0) #Add m0 elements de sítios em g;

    #Definindo  os arrays que guardará o grau, sítios e conjunto de graus e sítios da rede;
    DegreeFN = [] #Graus da nossa rede totalmente conectada terá m*t+m0 conexões FN: final nodes;
    VertexF = [] #Número total de sítios;

    K = [] #Lista que guardará os os valores dos sítios i, k vezes;
    K_supli = [] #Lista Suplementar, que juntará os novos elementos ao K;
    K_filter = [] #Lista que será filtrada para ser usada dentro do loop;
    #-----------------------------------------------------
    
    #------------------------------Criando a rede inicial-------------------------
    ver = np.zeros(N) #Guardará os sítios num array;

    for i in range(m0):
        ver[i] = int(g.vertex(i)) # Definindo todos valores dos sítios i como inteiros
        for j in range(i+1,m0):
            g.add_edge(g.vertex(i),g.vertex(j)) #Construindo uma rede totalmente ligada (inicial)
    
    #Gerando as listas com os sítios e ligações
    for i in range(m0):
        A = int(g.vertex(i).out_degree())
        B = int(g.vertex(i))
        DegreeFN.append(A) #Add na lista DegreeFN, guarda o grau de cada sítio i;
        VertexF.append(B) #Add na lista VerteX, guarda o sítio i, o valor em VerteX representa o sítio;
    
    
    for i in range(m0):
        K_supli = list(itertools.repeat(VertexF[i], DegreeFN[i])) #Gera uma  lista de elementos repetidos
        K += K_supli #Add essa lista de elementos repetidos em K
        
    #---------------------------------Crescimento da rede-------------------------------
    for i in range(m0,N):
        v = g.add_vertex()
        for j in range(m):
            j = randint(0, len(K_filter))
            u = g.vertex(K_filter[j])
            L = g.add_edge(v,u)
            #DegreeFN.append(int(u))
            #VertexF.append(int(u.out_degree()))
            K_supli = list(itertools.repeat(int(u), int(u.out_degree())))
            K += K_supli
            K_filter = list(filter(lambda val: val !=  u, K_filter))
        
        DegreeFN.append(int(g.vertex(i)))
        VertexF.append(int(g.vertex(i).out_degree()))
        K_supli = list(itertools.repeat(VertexF[i], DegreeFN[i]))
        K += K_supli
        K_filter = K
    #------------------------------------------------
    return K

In [11]:
print(barabasi_network(4,3,10,2))

ValueError: high <= 0